In [1]:
#Import necessary packages
import math
import numpy as np
np.set_printoptions(suppress=True,
                    formatter={'all': lambda x: '%5.3f' % x})
import pandas as pd
from scipy.optimize import brute, fmin, minimize
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['font.family'] = 'serif'

#Install import_ipynb so we can read dependenies as jupyter notebooks
!pip install ipynb

#Import dependencies
from ipynb.fs.full.FFT_option_valuation import H93_call_value_FFT
from ipynb.fs.full.CIR_calibration import CIR_calibration , r_list
from ipynb.fs.full.CIR_zcb_valuation import B

In [2]:
#Import CSV file with option data as a panda dataframe
raw = pd.read_csv('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/option_data.csv')

#Convert variables Maturity and Date to pandas datetime objects
raw['Maturity']= pd.to_datetime(raw['Maturity'], format='%Y-%m-%d') 
raw['Date']= pd.to_datetime(raw['Date'], format='%Y-%m-%d')

In [3]:
#Create a copy so we do not have to reload the data set if we start over
data = raw.copy()
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Maturity  159 non-null    datetime64[ns]
 1   Strike    159 non-null    float64       
 2   Call      159 non-null    float64       
 3   Date      159 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(2)
memory usage: 5.1 KB


,Maturity,Strike,Call,Date
0,2020-10-29,205.0,21.500,2020-10-26
1,2020-10-29,210.0,16.375,2020-10-26
2,2020-10-29,215.0,11.500,2020-10-26
3,2020-10-29,220.0,6.950,2020-10-26
4,2020-10-29,225.0,3.050,2020-10-26
...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26
155,2021-12-17,270.0,5.250,2020-10-26
156,2021-12-17,290.0,2.450,2020-10-26
157,2021-12-17,310.0,1.100,2020-10-26


In [4]:
#Create a variable, T, that shows how much, as a fraction of a year, that is left until maturity
data['diff'] = data['Maturity'] - data['Date']                     #number of days left to maturity
data['T']=(data['diff'].astype('timedelta64[D]').astype(int))/365  

#Drop unnecessary varibles
data.drop(['diff'], axis = 1, inplace=True)
options = data
options

,Maturity,Strike,Call,Date,T
0,2020-10-29,205.0,21.500,2020-10-26,0.008219
1,2020-10-29,210.0,16.375,2020-10-26,0.008219
2,2020-10-29,215.0,11.500,2020-10-26,0.008219
3,2020-10-29,220.0,6.950,2020-10-26,0.008219
4,2020-10-29,225.0,3.050,2020-10-26,0.008219
...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466
155,2021-12-17,270.0,5.250,2020-10-26,1.142466
156,2021-12-17,290.0,2.450,2020-10-26,1.142466
157,2021-12-17,310.0,1.100,2020-10-26,1.142466


In [5]:
#Initial short rate (Eonia 3.12.2020)
r0 = r_list[0]  

#Calibrate Short Rate Model to get calibrated estimates of CIR parameters used for zcb valuation
kappa_r, theta_r, sigma_r = CIR_calibration()

r = []
for row, option in options.iterrows():
    B0T = B([kappa_r, theta_r, sigma_r, r0, option['T']])
    r.append(-math.log(B0T) / option['T'])
options['r'] = r
options

Optimization terminated successfully.
         Current function value: 0.000020
         Iterations: 201
         Function evaluations: 382


,Maturity,Strike,Call,Date,T,r
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668
...,...,...,...,...,...,...
154,2021-12-17,250.0,10.150,2020-10-26,1.142466,0.000189
155,2021-12-17,270.0,5.250,2020-10-26,1.142466,0.000189
156,2021-12-17,290.0,2.450,2020-10-26,1.142466,0.000189
157,2021-12-17,310.0,1.100,2020-10-26,1.142466,0.000189


In [6]:
##################################################################################################################
################################################ Preliminaries ###################################################

#Define Stock price 
S0 = 226

#Slice the dataframe
#Default option selects all options in the csv file

#Choose only options that are within 2% of ATM 
#tol = 0.02  # percent ITM/OTM options
#options = [(np.abs(options['Strike'] - S0) / S0) < tol] 

#Choose specific strike prices 
# options = options[options['Strike'].isin([180, 200, 220, 240, 260])]

##################################################################################################################
######################################## Loss Function for the calibration #######################################

i = 0                                                              #Initialize iteration counter
min_MSE = 500                                                      #Initial min_MSE value
def H93_error_function_FFT(p0):
    ''' Error function for parameter calibration in BCC97 model via
    Lewis (2001) Fourier approach.

    Parameters
    ==========
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial, instantaneous variance

    Returns
    =======
    MSE: float
        mean squared error
    '''
    global i, min_MSE                                           #Set global variables 
    kappa_v, theta_v, sigma_v, rho, v0 = p0                     #Define initial values of the parameter vector p0
    if kappa_v < 0.0 or theta_v < 0.005 or sigma_v < 0.0 or rho < -1.0 or rho > 1.0:                       
        return 500.0
    if 2 * kappa_v * theta_v < sigma_v ** 2:      #Make sure that parameters adhere to their numerical boundaries
        return 500.0
    se = []
    for row, option in options.iterrows():
        model_value = H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0)
        se.append((model_value - option['Call']) ** 2)
    MSE = sum(se) / len(se)
    min_MSE = min(min_MSE, MSE)
    if i % 25 == 0:
        print('%4d |' % i, np.array(p0), '| %7.3f | %7.3f' % (MSE, min_MSE))
    i += 1
    return MSE

##################################################################################################################
################# Calibration of Heston (1993) parameters kappa_v, theta_v, sigma_v, rho, v0 #####################

def H93_calibration_full_FFT():
    ''' Calibrates H93 stochastic volatility model to market quotes. '''
    # first run with brute force
    # (scan sensible regions)
    p0 = brute(H93_error_function_FFT,
                ((2.5, 10.6, 5.0),  # kappa_v
                (0.01, 0.041, 0.01),  # theta_v
                (0.05, 0.251, 0.1),  # sigma_v
                (-0.75, 0.01, 0.25),  # rho
                (0.01, 0.031, 0.01)),  # v0
                finish=None)

    # second run with local, convex minimization
    # (dig deeper where promising)
    opt = fmin(H93_error_function_FFT, p0,  
                 xtol=0.001, ftol=0.001, 
                 maxiter=2000, maxfun=2000)
    np.save('C:/Users/1/Downloads/A_Python_in_financial_engineering/Project/opt_sv', np.array(opt))
    return opt

##################################################################################################################
################### Calculation of Heston (1993) option values with stochastic interest rates  ######################

def H93_calculate_model_values_FFT(p0):
    ''' Calculates all model values given parameter vector p0. '''
    kappa_v, theta_v, sigma_v, rho, v0 = p0  
    values = []
    for row, option in options.iterrows():
        model_value = H93_call_value_FFT(S0, option['Strike'], option['T'],
                            option['r'], kappa_v, theta_v, sigma_v, rho, v0)
        values.append(model_value)
    return np.array(values) 

##################################################################################################################
################################################# Run calibration ################################################

if __name__ == '__main__': 
    
    #Run and print the calibration
    opt = H93_calibration_full_FFT()
    print('')
    print("Calibrated parameter vector: ", opt)

    #Create pandas column for Heston (1993) call options values based on calibrated data
    options['Model'] = H93_calculate_model_values_FFT(opt)

   0 | [2.500000 0.010000 0.050000 -0.750000 0.010000] |  11.966 |  11.966
  25 | [2.500000 0.020000 0.050000 -0.750000 0.020000] |   7.013 |   6.503
  50 | [2.500000 0.020000 0.250000 -0.750000 0.030000] |   5.334 |   4.989
  75 | [2.500000 0.030000 0.150000 -0.500000 0.010000] |   7.850 |   3.880
 100 | [2.500000 0.040000 0.050000 -0.500000 0.020000] |   4.541 |   3.119
 125 | [2.500000 0.040000 0.250000 -0.500000 0.030000] |   3.161 |   3.119
 150 | [7.500000 0.010000 0.150000 -0.250000 0.010000] |  11.952 |   3.119
 175 | [7.500000 0.020000 0.050000 -0.250000 0.020000] |   6.990 |   3.119
 200 | [7.500000 0.020000 0.250000 -0.250000 0.030000] |   5.837 |   3.119
 225 | [7.500000 0.030000 0.150000 0.000000 0.010000] |   5.987 |   3.119
 250 | [7.500000 0.040000 0.050000 0.000000 0.020000] |   3.495 |   2.721
 275 | [7.500000 0.040000 0.250000 0.000000 0.030000] |   2.767 |   2.684
 300 | [7.399999 0.058747 0.202417 -0.448030 0.043284] |   1.697 |   1.669
 325 | [1.459294 0.028382 0.

In [7]:
#Drop max.rows option so we can see full dataframe 
pd.set_option('display.max_rows', None)
options

,Maturity,Strike,Call,Date,T,r,Model
0,2020-10-29,205.0,21.500,2020-10-26,0.008219,-0.004668,20.993430
1,2020-10-29,210.0,16.375,2020-10-26,0.008219,-0.004668,16.007220
2,2020-10-29,215.0,11.500,2020-10-26,0.008219,-0.004668,11.109436
3,2020-10-29,220.0,6.950,2020-10-26,0.008219,-0.004668,6.589491
4,2020-10-29,225.0,3.050,2020-10-26,0.008219,-0.004668,3.044258
5,2020-10-29,230.0,0.850,2020-10-26,0.008219,-0.004668,0.978724
6,2020-10-29,235.0,0.360,2020-10-26,0.008219,-0.004668,0.194899
7,2020-11-06,205.0,21.625,2020-10-26,0.030137,-0.004556,21.205659
8,2020-11-06,210.0,17.000,2020-10-26,0.030137,-0.004556,16.532750
9,2020-11-06,215.0,12.500,2020-10-26,0.030137,-0.004556,12.188521
